# 畳み込みニューラルネットワークの実装

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import sys,os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1,3,7,7)
col1 = im2col(x1,5,5,stride = 1,pad=0)
print(col1.shape)

x2 = np.random.rand(10,3,7,7)
col2 = im2col(x2,5,5,stride = 1,pad=0)
print(col2.shape)

x3 = np.random.rand(100,3,8,8)
col3 = im2col(x3,5,5,stride=1,pad=0)
print(col3.shape)

(9, 75)
(90, 75)
(1600, 75)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのモジュールを利用可能にするための設定
from common.util import im2col  # im2colをインポート（畳み込みを効率化する関数）

# Convolution（畳み込み層）のクラス定義
class convolution:
    def __init__(self, w, b, stride=1, pad=0):
        """
        コンストラクタ（初期化）
        w: 重み（フィルタ）→ 形状は (フィルタ数, 入力チャネル数, フィルタ高さ, フィルタ幅)
        b: バイアス → 形状は (フィルタ数,)
        stride: ストライド（フィルタをスライドする間隔）
        pad: パディング（入力データの周囲に付加するゼロの数）
        """
        self.w = w  # 重み（フィルタ）
        self.b = b  # バイアス
        self.stride = stride  # ストライド
        self.pad = pad  # パディング

    def forward(self, x):
        """
        順伝播（Forward）
        x: 入力データ → 形状は (バッチ数, 入力チャネル数, 高さ, 幅)
        """
        # 畳み込み後の出力の形状を計算
        FN, C, FH, FW = self.w.shape  # フィルタの形状（フィルタ数、チャネル数、高さ、幅）
        N, C, H, W = x.shape  # 入力データの形状（バッチ数、チャネル数、高さ、幅）
        out_h = int(1 + (H + 2 * self.pad - FH) / self.stride)  # 出力の高さ
        out_w = int(1 + (W + 2 * self.pad - FW) / self.stride)  # 出力の幅

        # 入力データをim2colで展開（パッチ化）
        col = im2col(x, FH, FW, self.stride, self.pad)
        # フィルタをim2colに対応する形状に変換
        col_w = self.w.reshape(FN, -1).T  # フィルタを2次元配列に変換

        # 行列積を計算（展開された入力データとフィルタ）
        out = np.dot(col, col_w) + self.b  # 畳み込み演算

        # 出力の形状を元に戻す（reshapeしてtranspose）
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)  # (バッチ数, フィルタ数, 出力高さ, 出力幅)
        return out

class Pooling:
    def __init__(self,pool_h,pool_w,stride=1,pad=0):
        self.pool_h=pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
    def forward(self,x):
        N,C,H,W=x.shape
        out_h=int(1+(H-self.pool_h)/self.stride)
        out_w=int(1+(W-self.pool_w)/self.stride)
        col = im2col(x,self.pool_h,self.pool_w,self.stride,self.pad)
        col = col.reshape(-1,self.pool_h*self.pool_w)

        out = np.max(col,axis=1)
        out = out.reshape(N,out_h,out_w,C).transpose(0,3,1,2)
        print(out.shape)
        return out


In [ ]:
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのモジュールを利用可能に
from dataset.mnist import load_mnist  # MNISTデータセットを読み込み
import numpy as np
from collections import OrderedDict  # 順序を保持した辞書型
from common.layers import Convolution, Relu, Pooling, Affine, SoftmaxWithLoss

class SimpleConvNet:
    """簡易的な畳み込みニューラルネットワークのクラス"""
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param={'filter_num': 30, 'filter_size': 5,
                             'pad': 0, 'stride': 1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        """
        ネットワークの初期化
        input_dim: 入力データの次元 (チャネル数, 高さ, 幅)
        conv_param: 畳み込み層のパラメータ（フィルタ数, フィルタサイズ, パディング, ストライド）
        hidden_size: 隠れ層のノード数
        output_size: 出力層のクラス数
        weight_init_std: 重みの初期化の標準偏差
        """
        filter_num = conv_param['filter_num']  # フィルタ数
        filter_size = conv_param['filter_size']  # フィルタサイズ
        filter_pad = conv_param['pad']  # パディングサイズ
        filter_stride = conv_param['stride']  # ストライド幅
        input_size = input_dim[1]  # 入力画像の高さ（幅も同じと仮定）

        # 畳み込み層とプーリング層を通過した後の出力サイズを計算
        conv_output_size = (input_size - filter_size + 2 * filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size / 2) * (conv_output_size / 2))

        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # レイヤの生成
        self.layers = OrderedDict()  # 順序付きディクショナリでレイヤを保持
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        """順伝播の処理"""
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    def loss(self, x, t):
        """損失関数を計算"""
        y = self.predict(x)  # 順伝播で出力を計算
        return self.lastLayer.forward(y, t)

    def gradient(self, x, t):
        """勾配を計算（誤差逆伝播法を利用）"""
        # 順伝播
        self.loss(x, t)

        # 逆伝播
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()  # レイヤを逆順にして逆伝播
        for layer in layers:
            dout = layer.backward(dout)

        # 勾配を格納
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db
        return grads

# ====== MNISTデータセットの読み込み ======
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

# リシェイプしてチャネル情報を追加
x_train = x_train.reshape(-1, 1, 28, 28)  # (バッチサイズ, チャネル数, 高さ, 幅)
x_test = x_test.reshape(-1, 1, 28, 28)

# ====== モデルの初期化 ======
network = SimpleConvNet()

# ====== 学習パラメータ ======
iters_num = 1000  # 繰り返し回数
train_size = x_train.shape[0]  # 訓練データの数
batch_size = 100  # ミニバッチのサイズ
learning_rate = 0.1  # 学習率

# 訓練の記録
train_loss_list = []

for i in range(iters_num):
    # ミニバッチの取得
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 勾配の計算
    grads = network.gradient(x_batch, t_batch)

    # パラメータの更新
    for key in network.params.keys():
        network.params[key] -= learning_rate * grads[key]

    # 損失関数の値を記録
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % 100 == 0:
        print(f"Iteration {i}: Loss = {loss:.4f}")

# ====== 訓練の可視化 ======
import matplotlib.pyplot as plt
plt.plot(train_loss_list)
plt.title("Training Loss")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()


Iteration 0: Loss = 2.3020
Iteration 100: Loss = 2.2789
